In [6]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
%matplotlib inline

In [7]:
def swc_to_dataframe( file_name ) :
    col_names = ['id', 'type', 'x', 'y', 'z', 'radius', 'parent', 'seg_id', 'level', 'mode', 'timestamp', 'TFresindex']
    df = pd.read_csv( file_name, sep=r"\s+", comment = "#", names = col_names, index_col = 'id')
    return df

In [9]:
#model_directory = "/allen/programs/celltypes/workgroups/mousecelltypes/_UPENN_fMOST"
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST"
swc_directory = os.path.join( model_directory,  "SWC_files", "SWCfile_67cells(Allen&WMU)_original")
image_directory = os.path.join( model_directory, "Inputs", "Original_Downsampled20um")

output_directory = os.path.join( model_directory, "SWC_files", "SWCfile_67cells(Allen&WMU)_original_rendered")

In [10]:
#
# component types
#
types = {}
types['soma'] = {'type_id': 1, 'color': 'black'}
types['axon'] = {'type_id': 2, 'color': 'cyan'}
types['basal dendrite'] = {'type_id': 3, 'color': 'orangered'}
types['apical dendrite'] = {'type_id': 4, 'color': 'maroon'}

#
# views for swc
#
views = {}
views['coronal'] = {'order': 0, 'x': 'y', 'y': 'x'}
views['sagittal'] = {'order': 1, 'x': 'z', 'y' : 'x'}
views['horizontal'] = {'order': 2, 'x': 'z', 'y' : 'y'}

#
# image slices
#
slices = {}
slices['coronal'] = {'swc': 'z', 'image' : 'z', 'transpose' : False }
slices['sagittal'] = {'swc': 'y', 'image' : 'x', 'transpose' : True }
slices['horizontal'] = {'swc' : 'x', 'image' : 'y', 'transpose' : True }

#
# scale from image pixel index to SWC coordinate
#
scales = {}
scales['z'] = 16
scales['x'] = 64
scales['y'] = 64

In [27]:
#
# Create morphology thumbnail for each SWC file in the swc directory
#
#
for swc_file in glob.glob(os.path.join(swc_directory, "*.swc")):

    bb = os.path.basename( swc_file )
    
    # find the associated brain file
    bname = bb.split('_')[0]
    image_file = os.path.join(image_directory, "mouse%sred_xy32z8.nii.gz" % bname )
    
    if not os.path.isfile( image_file ) :
        continue
        
    if bname != '182724' :
        continue
        
    # set the output thumbnail png file name
    cname = bb.split('.')[0]
    rendered_file = os.path.join( output_directory, "%s.nii.gz" % cname )
    
    print((cname,bname))
        
    #
    # read swc components into dataframe
    #
    df = swc_to_dataframe( swc_file )

    #
    # read in image file
    #
    image = sitk.ReadImage( image_file )

    #
    # scale swc coordinates to match image volume
    #
    for sk, sv in scales.items() :
        df[sk] = df[sk] / sv
    
    #
    # create output image
    #
    rendered = sitk.Image( image.GetSize(), sitk.sitkUInt8, 0 )
    rendered.CopyInformation( rendered )
    
    #
    # fill in rendered image
    #
    size = image.GetSize()
    for index, row in df.iterrows() :
        
        pixel = (round(row['y']),round(row['x']),round(row['z']))
        
        isOutside = False
        for i in range(3) :
            if pixel[i] >= size[i] :
                isOutside = True
                break
        
        if not isOutside :
            rendered.SetPixel( pixel, int(row['type']))
    
    
    #
    # save output file
    #
    sitk.WriteImage( rendered, rendered_file, True )
    
    

('182724_2238-X7096-Y20268', '182724')
('182724_2247-X6785-Y11803', '182724')
('182724_2333-X7081-Y21185', '182724')
('182724_2383-X9300-Y17890', '182724')
('182724_2494-X6419-Y20983', '182724')
('182724_2525-X8483-Y21088', '182724')
('182724_2526-X7529-Y21792', '182724')
('182724_2670-X5615-Y13225', '182724')
('182724_2926-X5279-Y13761', '182724')
('182724_2997-X5821-Y12613', '182724')
('182724_3848-X8903-Y23965', '182724')
('182724_3887-X5901-Y10339', '182724')
('182724_4327-X4847-Y18972', '182724')
('182724_4472-X5247-Y12162', '182724')
('182724_4580-X5627-Y9427', '182724')
('182724_4599-X8181-Y6776', '182724')
('182724_4785-X5002-Y11122', '182724')
('182724_5355-X5321-Y20589', '182724')
('182724_5410-X4393-Y11075', '182724')
('182724_5651-X9832-Y25682', '182724')
('182724_5793-X8695-Y5510', '182724')
('182724_5937-X3804-Y11955', '182724')
